# Change Detection Example: Log Ratio

This example shows how the Capella API can be used to fetch a time series stack of data, read data for a given bounding box directly from TileDB Cloud, and apply a log ratio change detection with an accumulator.

In [56]:
import json

from matplotlib import rcParams
from matplotlib import pyplot as plt

import numpy as np
import rasterio
from rasterio.plot import show
from rasterio.warp import transform_bounds
from rasterio.windows import Window
from rasterio.crs import CRS
from skimage import exposure
from scipy.ndimage import morphology
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance

np.seterr(divide='ignore', invalid='ignore')

%matplotlib inline

### Set up project variables

In [57]:
with open('filter.json') as f:
    filters = json.load(f)
    bbox = filters['bbox']

# Threshold setting for change detection
THRSET = 2 

# Windows sizes for filtering
MORPHWINSIZE = 3 # window size for Morphological filtering
FILTSIZE = 3 # window size for speckle filter

### Function to apply speckle filtering

In [58]:
def lee_filter(img, size):
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

### Use the API to search for Capella SAR data

In [59]:
result = ! rio capella --credentials credentials.json --area filter.json --collection rotterdam-aerial-mosaic --limit 50 query
fc = json.loads(result[0])
features = fc['features']

### Build a change heatmap from the time series

Ingests images two at a time by reading only the area within the bounding box from the cloud optimized geotiffs, speckle filters the images, performs log ratio change detection, thresholds and saves detection map into an accumulator, process repeats through all image pairs and builds a heatmap of change

In [60]:
# utility for reading clipped image and applying a lee despeckle filter
def clipped_read(fid):
    with rasterio.Env():
        with rasterio.open(fid) as src:        
            meta = src.meta
            native_bounds = transform_bounds(CRS.from_epsg(4326), src.crs, *bbox)        
            bounds_window = src.window(*native_bounds)
            bounds_window = bounds_window.intersection(Window(0, 0, src.width, src.height))

            img = src.read(1, window=bounds_window)

            img[img == meta['nodata']] = 0

            lee_filt_img = lee_filter(img, FILTSIZE)
            return lee_filt_img

In [ ]:
# Read files in two at a time and speckle filters
for i in range(0, len(features) - 1):
    ft1 = features[i]
    ft2 = features[i+1]
    fid1 = f"tiledb://capellaspace/{ft1['collection']}_{ft1['id']}"
    fid2 = f"tiledb://capellaspace/{ft2['collection']}_{ft2['id']}"
    
    img1 = clipped_read(fid1)
    img2 = clipped_read(fid2)
        
    # Calculate the log ratio of image pairs
    dIx = np.log(img2/img1)
    
    # Statistics and thresholding
    # Thresholding is empirically derived, requires manual adjustment of THRSET constant
    thr = np.nanmean(dIx) + THRSET*np.nanstd(dIx)
    dIx[dIx < thr] = 0.0
    dIx[dIx > thr] = 1.0

    # Morphological opening to reduce false alarms    
    w = (MORPHWINSIZE, MORPHWINSIZE)
    dIx = morphology.grey_opening(dIx, size=w)
    
    # Build accumulator
    if i == 0:
        cd = dIx
    else:
        cd += dIx

### Display the change detection result

In [ ]:
ft = features[0]

with rasterio.Env():
    fid = f"tiledb://capellaspace/{ft['collection']}_{ft['id']}"
    ci = clipped_read(fid)
    ci = exposure.adjust_log(ci, gain=10)

rcParams['figure.figsize'] = 10,5
fig, ax = plt.subplots(1,2)
ax[0].imshow(ci, 'gray');
ax[0].set_title("Context Image");
ax[1].imshow(cd);
ax[1].set_title("Change Detection Heatmap");